# Part 1: A Small Example

Investigating and testing the propagating flux ratio method

In [3]:
from fire_model import FireEquations, FireParams, Fuel
FireEquations.optimum_packing_ratio(5.0)

0.05364142973392205